# Propuesta Proyecto RecSys: Recomendación Grupal de Juegos de Mesa (setup)

Link de dataset: https://www.kaggle.com/datasets/threnjen/board-games-database-from-boardgamegeek?select=games.csv

In [2]:
!pip install -q kaggle
!pip install pyreclab
!pip install implicit
!pip install surprise
!pip install elliot
!pip install kagglehub

ERROR: Could not find a version that satisfies the requirement elliot (from versions: none)
ERROR: No matching distribution found for elliot


Tutorial usado: https://www.kaggle.com/discussions/general/74235

Tutorial adicional usado: https://www.youtube.com/watch?v=yEXkEUqK52Q

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("threnjen/board-games-database-from-boardgamegeek")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/threnjen/board-games-database-from-boardgamegeek/versions/4


Arriba de esto aparecerá un "path to dataset files", ese path se debe copiar y pegar en la línea de abajo:

In [4]:
path_to_dataset_files = path
#path_to_dataset_files = '/home/nico/.cache/kagglehub/datasets/threnjen/board-games-database-from-boardgamegeek/versions/4'
#path_to_dataset_files = '/root/.cache/kagglehub/datasets/threnjen/board-games-database-from-boardgamegeek/versions/4'

import os
# Guardamos el directorio actual
base_dir = os.getcwd()
# Cambiamos al directorio donde se encuentra el dataset
os.chdir(path_to_dataset_files)

# Importamos las librerias
import scipy.sparse as sparse
import pandas as pd
import numpy as np
import pyreclab
import tempfile
import implicit
import random
from surprise import accuracy

from abc import ABC, abstractmethod
import random

Generamos los datos a utilizar como un muestreo del dataset original pues es muy grande

In [5]:
# Leemos el csv y volvemos al directorio base del proyecto
user_ratings = pd.read_csv('user_ratings.csv')
mechanics_df = pd.read_csv('mechanics.csv')
# Volvemos al directorio base del proyecto
os.chdir(base_dir)
print(base_dir)

# Cambiamos username por un userid
a=list(set(list(user_ratings.Username)))
d = {a[i]: i for i in range(len(a))}
a_mod = [d[i] for i in list(user_ratings.Username)]
user_ratings["Username"] = a_mod

# Separamos training y testing
train     = list(set(user_ratings.Username))[:8000]
test      = list(set(user_ratings.Username))[8000:11000]
test_set  = user_ratings[user_ratings["Username"].isin(test)].sample(9000)
train_set = user_ratings[user_ratings["Username"].isin(train)].sample(35000)

# Generamos nuevo csv de training y testing
train_set.to_csv("train.csv", index=False, sep=',', header=True)
test_set.to_csv("test.csv", index=False, sep=',', header=True)

/content


# Creación de grupos

## Código importado

In [6]:
# Groups generator from: https://github.com/barnap/group-recommenders-offline-evaluation/blob/main/synthetic_groups_generation/groups_generators.py

class GroupsGenerator(ABC):

    @staticmethod
    def getGroupsGenerator(type):
        if type == "RANDOM":
            return RandomGroupsGenerator()
        elif type == "SIMILAR":
            return SimilarGroupsGenerator()
        elif type == "DIVERGENT":
            return DivergentGroupsGenerator()
        elif type == "SIMILAR_ONE_DIVERGENT":
            return MinorityGroupsGenerator()
        return None

    @staticmethod
    def compute_average_similarity(group, user_id_indexes, sim_matrix):
        similarities = list()
        for user_1 in group:
            user_1_index = user_id_indexes.tolist().index(user_1)
            for user_2 in group:
                user_2_index = user_id_indexes.tolist().index(user_2)
                if user_1 != user_2:
                    similarities.append(sim_matrix[user_1_index][user_2_index])
        return np.mean(similarities)

    @abstractmethod
    def generateGroups(self, user_id_indexes, user_id_set, similarity_matrix, group_sizes_to_create, group_number_to_create):
        pass


class RandomGroupsGenerator(GroupsGenerator):

    def generateGroups(self, user_id_indexes, user_id_set, similarity_matrix, group_sizes_to_create, group_number_to_create):
        groups_list = list()
        for group_size in group_sizes_to_create:
            for i in range(group_number_to_create):
                group = random.sample(user_id_set, group_size)
                groups_list.append(
                    {
                        "group_size": group_size,
                        "group_similarity": 'random',
                        "group_members": group,
                        "avg_similarity": GroupsGenerator.compute_average_similarity(group, user_id_indexes, similarity_matrix)
                    }
                )
            print(len(groups_list))
        return groups_list


class SimilarGroupsGenerator(GroupsGenerator):

    @staticmethod
    def select_user_for_sim_group(group, sim_matrix, user_id_indexes, sim_threshold=0.4):
        '''
        Helper function to the generate_similar_user_groups function. Given already selected group members, it randomly
        selects from the remaining users that has a PCC value >= sim_threshold to any of the existing members.
        :param group:
        :param sim_matrix:
        :param user_id_indexes:
        :param sim_threshold:
        :return:
        '''
        ids_to_select_from = set()
        for member in group:
            member_index = user_id_indexes.tolist().index(member)
            indexes = np.where(sim_matrix[member_index] >= sim_threshold)[0].tolist()
            user_ids = [user_id_indexes[index] for index in indexes]
            ids_to_select_from = ids_to_select_from.union(set(user_ids))
        candidate_ids = ids_to_select_from.difference(set(group))
        if len(candidate_ids) == 0:
            return None
        else:
            selection = random.sample(candidate_ids, 1)
            return selection[0]

    def generateGroups(self, user_id_indexes, user_id_set, similarity_matrix, group_sizes_to_create, group_number_to_create):
        groups_list = list()
        for group_size in group_sizes_to_create:
            groups_size_list = list()
            while (len(groups_size_list) < group_number_to_create):
                group = random.sample(user_id_set, 1)
                while len(group) < group_size:
                    new_member = SimilarGroupsGenerator.select_user_for_sim_group(group, similarity_matrix,
                                                                                  user_id_indexes,
                                                                                  sim_threshold=0.5)
                    if new_member is None:
                        break
                    group.append(new_member)
                if len(group) == group_size:
                    groups_size_list.append(
                        {
                            "group_size": group_size,
                            "group_similarity": 'similar',
                            "group_members": group,
                            "avg_similarity": GroupsGenerator.compute_average_similarity(group, user_id_indexes, similarity_matrix)
                        }
                    )
            groups_list.extend(groups_size_list)
            print(len(groups_list))
        return groups_list


class DivergentGroupsGenerator(GroupsGenerator):

    @staticmethod
    def select_user_for_divergent_group(group, sim_matrix, user_id_indexes, sim_threshold=0.0):
        '''
        Helper function to the generate_similar_user_groups function. Given already selected group members, it randomly
        selects from the remaining users that has a PCC value < sim_threshold to any of the existing members.
        :param group:
        :param sim_matrix:
        :param user_id_indexes:
        :param sim_threshold:
        :return:
        '''
        ids_to_select_from = set()
        for member in group:
            member_index = user_id_indexes.tolist().index(member)
            indexes = np.where(sim_matrix[member_index] < sim_threshold)[0].tolist()
            user_ids = [user_id_indexes[index] for index in indexes]
            ids_to_select_from = ids_to_select_from.union(set(user_ids))
        candidate_ids = ids_to_select_from.difference(set(group))
        if len(candidate_ids) == 0:
            return None
        else:
            selection = random.sample(candidate_ids, 1)
            return selection[0]

    def generateGroups(self, user_id_indexes, user_id_set, similarity_matrix, group_sizes_to_create, group_number_to_create):
        groups_list = list()
        for group_size in group_sizes_to_create:
            groups_size_list = list()
            while (len(groups_size_list) < group_number_to_create):
                group = random.sample(user_id_set, 1)
                while len(group) < group_size:
                    new_member = DivergentGroupsGenerator.select_user_for_divergent_group(group, similarity_matrix,
                                                                                     user_id_indexes,
                                                                                     sim_threshold=-0.1)
                    if new_member is None:
                        break
                    group.append(new_member)
                if len(group) == group_size:
                    groups_size_list.append(
                        {
                            "group_size": group_size,
                            "group_similarity": 'divergent',
                            "group_members": group,
                            "avg_similarity": GroupsGenerator.compute_average_similarity(group, user_id_indexes, similarity_matrix)
                        }
                    )
            groups_list.extend(groups_size_list)
            print(len(groups_list))
        return groups_list


class MinorityGroupsGenerator(GroupsGenerator):
    def generateGroups(self, user_id_indexes, user_id_set, similarity_matrix, group_sizes_to_create, group_number_to_create):
        groups_list = list()
        for group_size in group_sizes_to_create:
            groups_size_list = list()
            while (len(groups_size_list) < group_number_to_create):
                group = random.sample(user_id_set, 1)
                while len(group) < (group_size - 1):
                    new_member = SimilarGroupsGenerator.select_user_for_sim_group(group, similarity_matrix,
                                                                                     user_id_indexes,
                                                                                     sim_threshold=0.5)
                    if new_member is None:
                        break
                    group.append(new_member)

                dissimilar_member = DivergentGroupsGenerator.select_user_for_divergent_group(group, similarity_matrix,
                                                                                              user_id_indexes,
                                                                                              sim_threshold=-0.1)
                if dissimilar_member is not None:
                    group.append(dissimilar_member)
                if len(group) == group_size:
                    groups_size_list.append(
                        {
                            "group_size": group_size,
                            "group_similarity": 'similar_one_divergent',
                            "group_members": group,
                            "avg_similarity": GroupsGenerator.compute_average_similarity(group, user_id_indexes, similarity_matrix)
                        }
                    )
            groups_list.extend(groups_size_list)
            print(len(groups_list))
        return groups_list

## Código nuestro

Informacion para crear grupos

In [7]:
group_sizes_to_create = [4]        # [2, 3, 4, 5, 6, 7, 8]
group_similarity_to_create = "RANDOM"  # ["RANDOM", "SIMILAR", "DIVERGENT", "SIMILAR_ONE_DIVERGENT"]
group_number = 10000
group_number_te = 2000

Creacion de los grupos

In [8]:
# Extraccion de un sample para poder manejarlo
user_ratings = train_set.sample(5000)

# Informacion del dataset
user_matrix = user_ratings.pivot_table(columns='BGGId', index='Username', values='Rating')
user_id_set = set(user_ratings['Username'])
user_id_indexes = user_matrix.index.values
user_matrix = user_matrix.fillna(0)
numpy_array = user_matrix.to_numpy()
sim_matrix = np.corrcoef(numpy_array)

#Creacion del generador
grpGenerator = GroupsGenerator.getGroupsGenerator(group_similarity_to_create)
grpList = grpGenerator.generateGroups(user_id_indexes, user_id_set, sim_matrix, group_sizes_to_create, group_number)

#display(pd.DataFrame.from_records(grpList))
pd.DataFrame.from_records(grpList).to_csv('synthetic_groups.csv', index=False)

<ipython-input-6-beada3bb788c>:39: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  group = random.sample(user_id_set, group_size)


10000


Creacion de grupos de testing con archivo para replicabilidad

In [9]:
# Extraccion de un sample
testest_groups = pd.read_csv("userRatingTest.txt", sep=" ", names=["Username", "BGGId", "Rating"])
user_ratings_te = testest_groups.sample(5000)

# Informacion del dataset
user_matrix_te = user_ratings_te.pivot_table(columns='BGGId', index='Username', values='Rating')
user_id_set_te = set(user_ratings_te['Username'])
user_id_indexes_te = user_matrix_te.index.values
user_matrix_te = user_matrix_te.fillna(0)
numpy_array_te = user_matrix_te.to_numpy()
sim_matrix_te = np.corrcoef(numpy_array_te)

#Creacion del generador
grpGenerator_te = GroupsGenerator.getGroupsGenerator(group_similarity_to_create)
grpList_te = grpGenerator.generateGroups(user_id_indexes_te, user_id_set_te, sim_matrix_te, group_sizes_to_create, group_number_te)

#display(pd.DataFrame.from_records(grpList))
pd.DataFrame.from_records(grpList_te).to_csv('synthetic_groups_te.csv', index=False)

<ipython-input-6-beada3bb788c>:39: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  group = random.sample(user_id_set, group_size)


2000


# Evaluación de baselines para 1 usuario

In [10]:
# Trabajaremos con un top 10
top_n = 10
test_set.head(top_n)

,BGGId,Rating,Username
15394787,284108,6.0,9659
1439413,478,6.0,9765
7511941,205896,9.0,8428
8498015,233078,10.0,10041
5810723,172818,7.0,9465
16263784,45748,7.5,8749
6130766,131357,8.0,9552
18542198,91430,4.0,9724
5944263,257501,6.0,8428
17769744,92666,8.8,10315


In [9]:
# Revisemos el tamaño del dataset para asegurarnos de que tiene un tamaño trabajable:
print("Tamaño del dataset completo:", user_ratings.shape)
print("Tamaño del dataset de entrenamiento:", train_set.shape)
print("Tamaño del dataset de prueba:", test_set.shape)

Tamaño del dataset completo: (5000, 3)
Tamaño del dataset de entrenamiento: (35000, 3)
Tamaño del dataset de prueba: (9000, 3)


In [ ]:
# Evaluamos UserKNN
myUserKnn = pyreclab.UserKnn(dataset='train.csv', dlmchar=b',', header=False, usercol=2, itemcol=0, ratingcol=1)
myUserKnn.train(k=7, similarity='pearson')
_, maeUK, rmseUK = myUserKnn.test(input_file = 'test.csv', dlmchar = b',', header = False, usercol = 2, itemcol = 0, ratingcol = 1)
_, mapUK, ndcgUK = myUserKnn.testrec(input_file="test.csv", dlmchar=b',', header=False, usercol=2, itemcol=0, ratingcol=1, topn=top_n)

print(f"mae = {maeUK} y rmse = {rmseUK}")
print(f"map = {mapUK} y ndcg = {ndcgUK}")

In [ ]:
# Evaluamos ItemKNN
myItemKnn = pyreclab.ItemKnn(dataset='train.csv', dlmchar=b',', header=False, usercol=2, itemcol=0, ratingcol=1)
myItemKnn.train(k=7, similarity='pearson')
_, maeIK, rmseIK = myItemKnn.test(input_file = 'test.csv', dlmchar = b',', header = False, usercol = 2, itemcol = 0, ratingcol = 1)
_, mapIK, ndcgIK = myItemKnn.testrec(input_file="test.csv", dlmchar=b',', header=False, usercol=2, itemcol=0, ratingcol=1, topn=top_n)

print(f"mae = {maeIK} y rmse = {rmseIK}")
print(f"map = {mapIK} y ndcg = {ndcgIK}")

In [10]:
# Evaluamos SVD
mySVD = pyreclab.SVD(dataset='train.csv', dlmchar=b',', header=False, usercol=2, itemcol=0, ratingcol=1)
mySVD.train(factors=50, maxiter=80, lr=0.1, lamb=0.5)
_, maeSVD, rmseSVD = mySVD.test(input_file = 'test.csv', dlmchar = b',', header = False, usercol = 2, itemcol = 0, ratingcol = 1)
_, mapSVD, ndcgSVD = mySVD.testrec(input_file="test.csv", dlmchar=b',', header=False, usercol=2, itemcol=0, ratingcol=1, topn=top_n)

print(f"mae = {maeSVD} y rmse = {rmseSVD}")
print(f"map = {mapSVD} y ndcg = {ndcgSVD}")

mae = 7.200229080914603 y rmse = 7.200229080914603
map = 0.0003103068354794808 y ndcg = 0.00032204274683492846


In [11]:
# Evaluamos Most Popular
myMP = pyreclab.MostPopular(dataset='train.csv', dlmchar=b',', header=False, usercol=2, itemcol=0, ratingcol=1)
myMP.train(progress=False)
_, mapMP, ndcgMP = myMP.testrec(input_file="test.csv", dlmchar=b',', header=False, usercol=2, itemcol=0, ratingcol=1, topn=top_n)

print(f"map = {mapMP} y ndcg = {ndcgMP}")

map = 0.07503274712035485 y ndcg = 0.05025553932630089


In [12]:
# Evaluamos Random ratings
predictions = []

rating_scale = (1, 10)

for _, row in test_set.iterrows():
    itemId = row["BGGId"]; rating = row["Rating"]; userId = row["Username"]
    random_rating = random.uniform(rating_scale[0], rating_scale[1])
    predictions.append((userId, itemId, rating, random_rating, None))

accuracy.rmse(predictions)
accuracy.mae(predictions)


RMSE: 3.4129
MAE:  2.7831


2.783092934766391

# Recomendación multimodal para un usuario

Setup del metadata

In [11]:
!pip install lightfm

In [12]:
import ast
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, recall_at_k
from PIL import Image
import requests
from io import BytesIO

In [11]:

df = train_set
cols_with_id = {col: idx for idx, col in enumerate(mechanics_df.columns[1:])}

# Conjunto de features
item_styles = {}

for _, row in train_set.iterrows():
    bgg_id = row['BGGId']
    style_row = mechanics_df[mechanics_df['BGGId'] == bgg_id].drop(columns=['BGGId'])
    styles = style_row.columns[style_row.iloc[0] == 1].tolist()
    item_styles[bgg_id] = styles

print(item_styles)



itemslist = df['BGGId'].unique()
userslist = df['Username'].unique()
stylelist = [i for i in range(len(cols_with_id))]

{202755.0: ['Hand Management', 'Set Collection', 'Take That'], 1406.0: ['Auction/Bidding', 'Set Collection', 'Stock Holding', 'Negotiation', 'Trading', 'Income', 'Roll / Spin and Move', 'Lose a Turn', 'Player Elimination', 'Ownership', 'Track Movement', 'Loans'], 128621.0: ['Hand Management', 'Worker Placement'], 233961.0: ['Auction/Bidding', 'Trick-taking'], 339.0: ['Hand Management', 'Set Collection'], 2570.0: ['Auction/Bidding', 'Set Collection'], 8125.0: ['Area Majority / Influence', 'Auction/Bidding', 'Tile Placement', 'Negotiation', 'Commodity Speculation', 'Bribery'], 94.0: ['Stock Holding', 'Network and Route Building', 'Drafting'], 147020.0: ['Hand Management', 'Take That', 'Drafting', 'Delayed Purchase', 'Deck, Bag, and Pool Building'], 169124.0: ['Team-Based Game', 'Pick-up and Deliver', 'Dexterity'], 233867.0: ['Simultaneous Action Selection', 'Pattern Building', 'Paper-and-Pencil', 'Bingo', 'Solo / Solitaire Game', 'Contracts'], 85.0: ['Dice Rolling', 'Point to Point Movem

In [12]:
#Conjunto de features, pero numericos

item_styles_with_ids = {}
for item_id, styles in item_styles.items():
    style_ids = [cols_with_id[style] for style in styles]
    item_styles_with_ids[item_id] = style_ids
print(item_styles_with_ids)

{202755.0: [4, 9, 62], 1406.0: [2, 9, 15, 24, 25, 27, 31, 36, 42, 101, 104, 134], 128621.0: [4, 95], 233961.0: [2, 6], 339.0: [4, 9], 2570.0: [2, 9], 8125.0: [1, 2, 10, 24, 48, 92], 94.0: [15, 21, 106], 147020.0: [4, 62, 106, 135, 136], 169124.0: [63, 73, 155], 233867.0: [5, 18, 41, 75, 87, 147], 85.0: [3, 22, 45, 59], 24508.0: [10, 20, 28, 57, 58, 63], 11.0: [4, 9, 23, 24, 25], 31552.0: [5, 33, 42, 45], 472.0: [3, 10, 26, 32, 43, 156], 478.0: [9, 32, 36, 50, 54, 69, 78, 106, 107], 194607.0: [26, 57, 136], 203993.0: [3, 32, 95, 106, 107], 2266.0: [18, 19, 55], 141572.0: [49, 106, 135, 136], 113924.0: [3, 4, 11, 20, 32, 35, 38, 42, 49], 192458.0: [4, 32, 46, 95, 106, 107], 5349.0: [3, 7], 188.0: [14, 17], 178900.0: [26, 50, 55, 63, 128], 13.0: [3, 7, 20, 21, 24, 25, 27, 28, 29, 30], 250870.0: [9], 131357.0: [32, 42, 50, 54, 55, 62, 78], 189067.0: [3, 9, 62, 95], 110327.0: [9, 60, 62, 78, 95, 101, 147], 557.0: [], 158899.0: [4, 32, 33, 37, 55, 62, 150], 2550.0: [49, 122], 156786.0: [4, 1

In [13]:
interactions = [(row['Username'], row['BGGId'], row['Rating']) for index, row in df.iterrows()]

In [14]:
dataset = Dataset()
all_features = set(feature for features in item_styles.values() for feature in features)

dataset.fit(users=userslist, items=itemslist, item_features=all_features)

In [15]:
(interactions_matrix, weights_matrix) = dataset.build_interactions(
    ((x[0], x[1], x[2]) for x in interactions)
)

item_features = dataset.build_item_features(
    ((item_id, features) for item_id, features in item_styles.items())
)
print(interactions_matrix)
# print(item_features)

  (0, 0)	1
  (1, 1)	1
  (2, 2)	1
  (3, 3)	1
  (4, 4)	1
  (5, 5)	1
  (6, 6)	1
  (7, 7)	1
  (8, 8)	1
  (9, 9)	1
  (10, 10)	1
  (5, 11)	1
  (11, 12)	1
  (12, 13)	1
  (13, 14)	1
  (14, 15)	1
  (15, 16)	1
  (16, 17)	1
  (17, 18)	1
  (18, 19)	1
  (19, 20)	1
  (20, 21)	1
  (21, 8)	1
  (22, 22)	1
  (23, 23)	1
  :	:
  (4925, 192)	1
  (1072, 635)	1
  (1899, 1189)	1
  (351, 1124)	1
  (1857, 127)	1
  (1236, 7563)	1
  (524, 905)	1
  (854, 420)	1
  (4453, 307)	1
  (2208, 1677)	1
  (1054, 381)	1
  (870, 870)	1
  (4926, 773)	1
  (1328, 416)	1
  (1422, 7564)	1
  (1067, 118)	1
  (4927, 254)	1
  (102, 725)	1
  (1193, 2099)	1
  (3074, 1149)	1
  (272, 353)	1
  (2607, 56)	1
  (837, 252)	1
  (763, 3982)	1
  (2269, 634)	1


In [16]:
model = LightFM(no_components=30, loss='warp')
model.fit(interactions_matrix, item_features=item_features, epochs=10, num_threads=4)

In [17]:
def recommend(model, dataset, user_ids, n_items=5):
    n_users, n_items_total = interactions_matrix.shape
    item_ids = np.arange(n_items_total)
    recommendations_per_user = {}

    for user_id in user_ids:
        scores = model.predict(user_id, item_ids, item_features=item_features)
        top_items = item_ids[np.argsort(-scores)][:n_items]
        item_mapping = dataset.mapping()[2]
        item_id_mapping = {v: k for k, v in item_mapping.items()}
        recommended_items = [item_id_mapping[item] for item in top_items]
        print(f"User {user_id} recommended items: {recommended_items}")
        recommendations_per_user[user_id] = recommended_items

    return recommendations_per_user


In [18]:
train_precision = precision_at_k(model, interactions_matrix, item_features=item_features, k=5).mean()
train_recall = recall_at_k(model, interactions_matrix, item_features=item_features, k=5).mean()

print(f'Train precision at k: {train_precision}')
print(f'Train Recall: {train_recall}')

Train precision at k: 0.061323054134845734
Train Recall: 0.11931285594531489


In [19]:
recommendations_group = recommend(model, dataset, [1, 2, 3], n_items=1000)
print(type(userslist))
print(recommendations_group)


User 1 recommended items: [171, 188, 2452, 2083, 624, 3076, 521, 382, 2651, 320, 432, 9220, 903, 37196, 5048, 1406, 2397, 45, 5232, 171129, 20100, 2842, 2136, 108687, 2389, 39856, 483, 3728, 41114, 4425, 150, 153938, 318985, 194655, 31999, 2281, 150145, 515, 160477, 188834, 2386, 40990, 5432, 1613, 478, 181, 12942, 143884, 592, 13, 555, 2249, 173, 1602, 4209, 8098, 14254, 19841, 6795, 128882, 3071, 2679, 1410, 822, 63268, 28023, 91514, 110327, 6887, 11, 1465, 120677, 327, 31358, 3955, 2065, 272380, 2346, 464, 2223, 38159, 19237, 26566, 143147, 236217, 30539, 2322, 619, 215, 949, 19857, 197405, 42490, 24068, 156129, 66589, 39463, 148228, 24508, 1258, 37904, 27172, 5251, 24396, 92415, 143741, 172, 28720, 311193, 403, 7805, 10093, 4636, 1294, 41, 8217, 27532, 1501, 8195, 129622, 8203, 2173, 9441, 31481, 3662, 62871, 70323, 2448, 5072, 1960, 165722, 220, 103343, 38054, 1382, 218530, 5494, 1001, 2921, 31730, 24181, 1917, 33604, 35497, 2943, 83195, 227072, 283355, 1232, 2944, 847, 118, 2593,

---
---

# Ponderación para grupos de usuarios con recomendaciones de metadata

In [13]:
groups_df = pd.read_csv('synthetic_groups.csv')
groups_df.head()

,group_size,group_similarity,group_members,avg_similarity
0,4,random,"[1586, 2813, 5274, 3049]",-0.000556
1,4,random,"[6078, 2802, 2140, 62]",-0.000407
2,4,random,"[4320, 3595, 7441, 1632]",-0.000670
3,4,random,"[6712, 2983, 6394, 863]",-0.000717
4,4,random,"[3516, 212, 5313, 3732]",-0.000608


Código adaptado de repositorio mencionado anteriormente, en este caso tomamos los items que en promedio son más preferidos para recomendarlos al grupo.

In [25]:
def recommend_for_group(model, dataset, group_members, n_items=5):
    n_users, n_items_total = interactions_matrix.shape
    item_ids = np.arange(n_items_total)
    user_mapping = dataset.mapping()[0]

    group_scores = []
    for user_id in group_members:
        try:
            internal_user_id = user_mapping[user_id]
            scores = model.predict(internal_user_id, item_ids, item_features=item_features)
            group_scores.append(scores)
        except KeyError:
            continue
        except Exception:
            continue

    if not group_scores:
        return []

    average_scores = np.mean(group_scores, axis=0)
    top_items = item_ids[np.argsort(-average_scores)][:n_items]

    item_mapping = dataset.mapping()[2]
    item_id_mapping = {v: k for k, v in item_mapping.items()}
    recommended_items = [item_id_mapping[item] for item in top_items]

    return recommended_items

group_recommendations = []
for _, row in groups_df.iterrows():
    group_members = eval(row['group_members']) if isinstance(row['group_members'], str) else row['group_members']
    recommendations = recommend_for_group(model, dataset, group_members, n_items=10)
    group_recommendations.append({
        'group_members': group_members,
        'recommendations': recommendations
    })

recommendations_df = pd.DataFrame(group_recommendations)
recommendations_df.to_csv('group_recommendations.csv', index=False)

In [ ]:
def recommend_for_group_multiple_models(model_name, model, group_members, n_items=10, include_rated=False):
    """
    Generate recommendations for a group using different pyreclab models
    Following exact format of original recommend_for_group function
    """
    group_scores = []


    for user_id in group_members:
        #print(f"Recomendacion para user {user_id}")
        try:
            ranking = model.recommend(str(user_id), n_items, include_rated)

            recommended_items = ranking
            group_scores.append(recommended_items)

        except Exception as e:
            print(f"Error getting recommendations for user {user_id}: {str(e)}")
            continue

    if not group_scores:
        return []

    item_counts = {}
    for user_recs in group_scores:
        for item in user_recs:
            item_counts[item] = item_counts.get(item, 0) + 1

    sorted_items = sorted(item_counts.items(), key=lambda x: x[1], reverse=True)
    recommendations = [int(item) for item, _ in sorted_items[:n_items]]

    return recommendations

models = {
    'SVD': mySVD,
    # 'UserKNN': myUserKnn,
    # 'ItemKNN': myItemKnn,
    'MostPopular': myMP
}

for model_name, model in models.items():
    print(f"\nGenerating recommendations using {model_name}...")
    group_recommendations = []

    for _, row in groups_df.head(1000).iterrows():
        group_members = ast.literal_eval(row['group_members']) if isinstance(row['group_members'], str) else row['group_members']
        recommendations = recommend_for_group_multiple_models(model_name, model, group_members, n_items=10)

        group_recommendations.append({
            'group_members': str(group_members),
            'recommendations': str(recommendations)
        })

    recommendations_df = pd.DataFrame(group_recommendations)
    recommendations_df.to_csv(f'{model_name.lower()}_group_recommendations.csv', index=False)
    print(f"Generated recommendations for {len(recommendations_df)} groups")
    print("Sample format:")
    print(recommendations_df.head(1))

In [26]:
def parse_group_data(groups_df):
    parsed_data = []

    for _, row in groups_df.head(30).iterrows():
        group_members = ast.literal_eval(row['group_members'])
        recommendations = ast.literal_eval(row['recommendations'])
        parsed_data.append((group_members, recommendations))

    print(f"\nTotal groups parsed: {len(parsed_data)}")
    return parsed_data

def calculate_relevance(train_df, user_id, item_id):
    rating = train_df[
        (train_df['Username'] == user_id) &
        (train_df['BGGId'] == item_id)
    ]['Rating']

    return rating.iloc[0] if not rating.empty else 0

def calculate_group_metrics(parsed_data, train_df, k=5, threshold=3.5):
    total_precision = 0
    total_recall = 0
    total_ndcg = 0
    total_relevance = 0
    total_groups = len(parsed_data)
    total_hits = 0
    total_possible_hits = 0
    all_group_sizes = []

    for i, (group_members, recommendations) in enumerate(parsed_data):
        print(f"\rProcessing Group {i+1}/{total_groups}", end="")
        recommended_items = recommendations[:k]
        group_relevance = []
        group_hits = 0
        for item_id in recommended_items:
            member_ratings = []
            for user_id in group_members:
                rating = calculate_relevance(train_df, user_id, item_id)
                if rating > 0:
                    member_ratings.append(rating)
                    group_hits += 1
                total_possible_hits += 1
            avg_rating = np.mean(member_ratings) if member_ratings else 0
            group_relevance.append(avg_rating)

        total_hits += group_hits

        binary_relevance = [1 if rel >= threshold else 0 for rel in group_relevance]

        total_relevant = 0
        unique_items = train_df['BGGId'].unique()
        for item_id in unique_items:
            member_ratings = []
            for user_id in group_members:
                rating = calculate_relevance(train_df, user_id, item_id)
                if rating > 0:
                    member_ratings.append(rating)
            avg_rating = np.mean(member_ratings) if member_ratings else 0
            if avg_rating >= threshold:
                total_relevant += 1

        precision = sum(binary_relevance) / k if k > 0 else 0
        recall = sum(binary_relevance) / total_relevant if total_relevant > 0 else 0

        dcg = sum((2 ** rel - 1) / np.log2(idx + 2)
                  for idx, rel in enumerate(group_relevance))
        ideal_relevance = sorted(group_relevance, reverse=True)
        idcg = sum((2 ** rel - 1) / np.log2(idx + 2)
                   for idx, rel in enumerate(ideal_relevance))
        ndcg = dcg / idcg if idcg > 0 else 0

        total_precision += precision
        total_recall += recall
        total_ndcg += ndcg
        total_relevance += np.mean(group_relevance) if group_relevance else 0

    avg_metrics = {
        "average_precision": total_precision / total_groups,
        "average_recall": total_recall / total_groups,
        "average_ndcg": total_ndcg / total_groups,
        "average_relevance": total_relevance / total_groups,
        # "hit_rate": total_hits / total_possible_hits,
        # "average_group_size": np.mean(all_group_sizes),
        # "total_groups": total_groups,
        # "total_hits": total_hits,
        # "total_possible_hits": total_possible_hits
    }

    return avg_metrics

# train_df = pd.read_csv('train.csv')
# groups_df = pd.read_csv('group_recommendations.csv')
# parsed_data = parse_group_data(groups_df)
# metrics = calculate_group_metrics(parsed_data, train_df, k=5)
# print(f"\nPrecision: {metrics['average_precision']:.3f}")
# print(f"Recall: {metrics['average_recall']:.3f}")
# print(f"nDCG: {metrics['average_ndcg']:.3f}")
# print(f"Relevance Score: {metrics['average_relevance']:.2f}")

# train_df = pd.read_csv('train.csv')
# groups_df = pd.read_csv('svd_group_recommendations.csv')
# parsed_data = parse_group_data(groups_df)
# metrics = calculate_group_metrics(parsed_data, train_df, k=5)
# print(f"\nPrecision: {metrics['average_precision']:.3f}")
# print(f"Recall: {metrics['average_recall']:.3f}")
# print(f"nDCG: {metrics['average_ndcg']:.3f}")
# print(f"Relevance Score: {metrics['average_relevance']:.2f}")

# train_df = pd.read_csv('train.csv')
# groups_df = pd.read_csv('mostpopular_group_recommendations.csv')
# parsed_data = parse_group_data(groups_df)
# metrics = calculate_group_metrics(parsed_data, train_df, k=5)
# print(f"\nPrecision: {metrics['average_precision']:.3f}")
# print(f"Recall: {metrics['average_recall']:.3f}")
# print(f"nDCG: {metrics['average_ndcg']:.3f}")
# print(f"Relevance Score: {metrics['average_relevance']:.2f}")

# Hasta aca esta probado

In [ ]:
# New cell for AGREE implementation
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from ast import literal_eval
from scipy import sparse

# Model classes
class UserEmbeddingLayer(nn.Module):
    def __init__(self, num_users, embedding_dim):
        super(UserEmbeddingLayer, self).__init__()
        self.userEmbedding = nn.Embedding(num_users, embedding_dim)

    def forward(self, user_inputs):
        return self.userEmbedding(user_inputs)

class ItemEmbeddingLayer(nn.Module):
    def __init__(self, num_items, embedding_dim):
        super(ItemEmbeddingLayer, self).__init__()
        self.itemEmbedding = nn.Embedding(num_items, embedding_dim)

    def forward(self, item_inputs):
        return self.itemEmbedding(item_inputs)

class GroupEmbeddingLayer(nn.Module):
    def __init__(self, num_groups, embedding_dim):
        super(GroupEmbeddingLayer, self).__init__()
        self.groupEmbedding = nn.Embedding(num_groups, embedding_dim)

    def forward(self, group_inputs):
        return self.groupEmbedding(group_inputs)

class AttentionLayer(nn.Module):
    def __init__(self, embedding_dim, drop_ratio=0.1):
        super(AttentionLayer, self).__init__()
        self.linear = nn.Sequential(
            nn.Linear(embedding_dim, 16),
            nn.ReLU(),
            nn.Dropout(drop_ratio),
            nn.Linear(16, 1)
        )

    def forward(self, x):
        out = self.linear(x)
        weight = torch.softmax(out.view(1, -1), dim=1)
        return weight

class PredictLayer(nn.Module):
    def __init__(self, embedding_dim, drop_ratio=0.1):
        super(PredictLayer, self).__init__()
        self.linear = nn.Sequential(
            nn.Linear(embedding_dim, 8),
            nn.ReLU(),
            nn.Dropout(drop_ratio),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        return self.linear(x)

class AGREE(nn.Module):
    def __init__(self, num_users, num_items, num_groups, embedding_dim, group_member_dict, drop_ratio=0.1):
        super(AGREE, self).__init__()
        self.userembeds = UserEmbeddingLayer(num_users, embedding_dim)
        self.itemembeds = ItemEmbeddingLayer(num_items, embedding_dim)
        self.groupembeds = GroupEmbeddingLayer(num_groups, embedding_dim)
        self.attention = AttentionLayer(2 * embedding_dim, drop_ratio)
        self.predictlayer = PredictLayer(3 * embedding_dim, drop_ratio)
        self.group_member_dict = group_member_dict

        # Initialize weights
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight)
            if isinstance(m, nn.Embedding):
                nn.init.xavier_normal_(m.weight)

    def forward(self, group_inputs, user_inputs, item_inputs):
        if (group_inputs is not None) and (user_inputs is None):
            return self.group_forward(group_inputs, item_inputs)
        else:
            return self.user_forward(user_inputs, item_inputs)

    def group_forward(self, group_inputs, item_inputs):
        group_embeds = torch.Tensor()
        item_embeds_full = self.itemembeds(torch.LongTensor(item_inputs))

        for i, j in zip(group_inputs, item_inputs):
            members = self.group_member_dict[i.item()]
            members_embeds = self.userembeds(torch.LongTensor(members))
            items_numb = [j] * len(members)
            item_embeds = self.itemembeds(torch.LongTensor(items_numb))

            group_item_embeds = torch.cat((members_embeds, item_embeds), dim=1)
            at_wt = self.attention(group_item_embeds)
            g_embeds_with_attention = torch.matmul(at_wt, members_embeds)
            group_embeds_pure = self.groupembeds(torch.LongTensor([i]))
            g_embeds = g_embeds_with_attention + group_embeds_pure
            group_embeds = torch.cat((group_embeds, g_embeds))

        element_embeds = torch.mul(group_embeds, item_embeds_full)
        new_embeds = torch.cat((element_embeds, group_embeds, item_embeds_full), dim=1)
        return torch.sigmoid(self.predictlayer(new_embeds))

    def user_forward(self, user_inputs, item_inputs):
        user_embeds = self.userembeds(user_inputs)
        item_embeds = self.itemembeds(item_inputs)
        element_embeds = torch.mul(user_embeds, item_embeds)
        new_embeds = torch.cat((element_embeds, user_embeds, item_embeds), dim=1)
        return torch.sigmoid(self.predictlayer(new_embeds))

# Load and prepare data
groups_df = pd.read_csv('synthetic_groups.csv').iloc[:10000]
groups_df['group_members'] = groups_df['group_members'].apply(literal_eval)

# Convert sparse matrix to dense or CSR format for indexing
if sparse.issparse(interactions_matrix):
    interactions_matrix = interactions_matrix.tocsr()

# Create mappings and initialize model
all_users = set()
for members in groups_df['group_members']:
    all_users.update(members)

user_to_idx = {old_id: idx for idx, old_id in enumerate(sorted(all_users))}
item_to_idx = {old_id: idx for idx, old_id in enumerate(range(interactions_matrix.shape[1]))}
group_member_dict = {
    group_id: [user_to_idx[user] for user in members]
    for group_id, members in enumerate(groups_df['group_members'])
}

# Initialize model
embedding_dim = 32
model = AGREE(
    num_users=len(user_to_idx),
    num_items=len(item_to_idx),
    num_groups=len(groups_df),
    embedding_dim=embedding_dim,
    group_member_dict=group_member_dict
)

# Train model
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

print("Starting training...")
for epoch in range(5):  # 5 epochs as example
    break
    #print(f"Epoch: {epoch+1}")
    model.train()
    total_loss = 0

    # Train on individual interactions
    for user_idx in range(interactions_matrix.shape[0]):
        #print("New user")
        # Get non-zero elements for this user
        items = interactions_matrix[user_idx].nonzero()[1]

        for item_idx in items:
            #print("New item")
            user_tensor = torch.LongTensor([user_idx])
            item_tensor = torch.LongTensor([item_idx])

            prediction = model(None, user_tensor, item_tensor)
            target = torch.FloatTensor([[1.0]])  # Changed to 2D tensor

            loss = criterion(prediction, target)

            # Negative sampling
            neg_item_idx = np.random.randint(0, interactions_matrix.shape[1])
            while interactions_matrix[user_idx, neg_item_idx] != 0:
                neg_item_idx = np.random.randint(0, interactions_matrix.shape[1])

            neg_item_tensor = torch.LongTensor([neg_item_idx])
            neg_prediction = model(None, user_tensor, neg_item_tensor)
            neg_target = torch.FloatTensor([[0.0]])  # Changed to 2D tensor

            loss += criterion(neg_prediction, neg_target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

    print(f'Epoch {epoch+1}/5, Loss: {total_loss}')

print("Training completed!")

# Function to get recommendations for a group
def get_group_recommendations(model, group_idx, num_items, top_k=10):
    model.eval()
    with torch.no_grad():
        scores = []
        group_tensor = torch.LongTensor([group_idx])

        for item_idx in range(num_items):
            item_tensor = torch.LongTensor([item_idx])
            score = model(group_tensor, None, item_tensor)
            scores.append((item_idx, score.item()))

        # Get top-k recommendations
        recommendations = sorted(scores, key=lambda x: x[1], reverse=True)[:top_k]
        return [item_idx for item_idx, _ in recommendations]

# Get recommendations for first group
group_idx = 0
recommendations = get_group_recommendations(model, group_idx, interactions_matrix.shape[1])
print(f"\nTop 10 recommendations for group {group_idx}:")
print(recommendations)

# Optional: Get recommendations for all groups
print("\nGenerating recommendations for all groups...")
all_group_recommendations = {}
for group_idx in range(len(groups_df)):
    recommendations = get_group_recommendations(model, group_idx, interactions_matrix.shape[1])
    all_group_recommendations[group_idx] = recommendations
    print(f"Group {group_idx}: {recommendations}")

Creamos archivos en el formato que usa AGREE

In [13]:
with open("negative_info.txt", "r") as fff:
    info = fff.read().strip().split("\n")
    info = [i.split(" + ") for i in info]
    dict_neg = {a: b for a, b in info}

with open("train.csv", "r") as fff:
    info_tr = fff.read().strip().split("\n")
    info_tr = [i.split(",") for i in info_tr[1:]]
    with open("userRatingTrain.txt", "w") as aaa:
        for line in info_tr:
            aaa.write(f"{line[2]} {line[0]} {int(float(line[1])*10)}\n")

with open("userRatingTest.txt", "r") as aaa:
    info_te = aaa.read().strip().split("\n")
    info_te = [i.split(" ") for i in info_te]
    with open("userRatingNegative.txt", "w") as bbb:
        for i in range(len(info_te)):
            bbb.write(f"({info_te[i][0]},{info_te[i][1]}) {dict_neg[info_te[i][0]]}\n")
            if i == 1000:
                break

dict_te = {user: [data for data in info_te if data[0] == user] for user in set(i[0] for i in info_te)}
dict_tr = {user: [data for data in info_tr if data[0] == user] for user in set(i[0] for i in info_tr)}

with open("synthetic_groups.csv", "r") as fff:
    with open("synthetic_groups_te.csv", "r") as fff1:
        info1 = fff.read().strip().split("\n")
        info2 = fff1.read().strip().split("\n")
        info1 = [(i[:10].replace(",",";") + i[10:i.index(']')] + i[i.index(']'):].replace(",",";")).split(";") for i in info1[1:]]
        info2 = [(i[:10].replace(",",";") + i[10:i.index(']')] + i[i.index(']'):].replace(",",";")).split(";") for i in info2[1:]]
        info3 = info1 + info2
        with open("groupMember.txt", "w") as aaa:
            with open("groupRatingTrain.txt", "w") as bbb:
                with open("groupRatingTest.txt", "w") as ccc:
                    with open("groupRatingNegative.txt", "w") as ddd:
                        for idx, line in enumerate(info1):
                            aaa.write(f"{idx} {line[2][2:-2].replace(' ','')}\n")
                            for user in line[2][2:-2].replace(' ','').split(","):
                                try:
                                    for rating in dict_tr[user]:
                                        bbb.write(f"{idx} {rating[2]} {int(float(rating[1])*10)}\n")
                                except KeyError:
                                    continue

                        for idxx, line in enumerate(info2):
                            if idxx == 1000:
                              break
                            aaa.write(f"{idx + idxx + 1} {line[2][2:-2].replace(' ','')}\n")
                            for user in line[2][2:-2].replace(' ','').split(","):
                                for rating in dict_te[user]:
                                  ccc.write(f"{idx + idxx + 1} {rating[1]} {rating[2]}\n")
                                  ddd.write(f"({idx + idxx + 1},{rating[1]}) {dict_neg[rating[0]]}\n")


Archivo config.py

In [16]:
'''
Created on Nov 10, 2017
Store parameters

@author: Lianhai Miao
'''

class Config(object):
    def __init__(self):
        self.path = ''
        self.user_dataset = self.path + 'userRating'
        self.group_dataset = self.path + 'groupRating'
        self.user_in_group_path = "groupMember.txt"
        self.embedding_size = 32
        self.epoch = 30
        self.num_negatives = 6
        self.batch_size = 15
        self.lr = [0.000005, 0.000001, 0.0000005]
        self.drop_ratio = 0.2
        self.topK = 5

Archivo utils.py

In [17]:
'''
Created on Nov 10, 2017
Deal something

@author: Lianhai Miao
'''
import torch
from torch.autograd import Variable
import numpy as np
import math
import heapq

class Helper(object):
    """
        utils class: it can provide any function that we need
    """
    def __init__(self):
        self.timber = True

    def gen_group_member_dict(self, path):
        g_m_d = {}
        with open(path, 'r') as f:
            line = f.readline().strip()
            while line != None and line != "":
                a = line.split(' ')
                g = int(a[0])
                g_m_d[g] = []
                for m in a[1].split(','):
                    g_m_d[g].append(int(m))
                line = f.readline().strip()
        return g_m_d

    def evaluate_model(self, model, testRatings, testNegatives, K, type_m):
        """
        Evaluate the performance (Hit_Ratio, NDCG) of top-K recommendation
        Return: score of each test rating.
        """
        hits, ndcgs = [], []

        for idx in range(len(testRatings)):
            (hr,ndcg) = self.eval_one_rating(model, testRatings, testNegatives, K, type_m, idx)
            hits.append(hr)
            ndcgs.append(ndcg)
        return (hits, ndcgs)


    def eval_one_rating(self, model, testRatings, testNegatives, K, type_m, idx):
        rating = testRatings[idx]
        items = testNegatives[idx]
        u = rating[0]
        gtItem = rating[1]
        items.append(gtItem)
        # Get prediction scores
        map_item_score = {}
        users = np.full(len(items), u)

        users_var = torch.from_numpy(users)
        users_var = users_var.long()
        items_var = torch.LongTensor(items)
        if type_m == 'group':
            predictions = model(users_var, None, items_var)
        elif type_m == 'user':
            predictions = model(None, users_var, items_var)
        for i in range(len(items)):
            item = items[i]
            map_item_score[item] = predictions.data.numpy()[i]
        items.pop()

        # Evaluate top rank list
        ranklist = heapq.nlargest(K, map_item_score, key=map_item_score.get)
        hr = self.getHitRatio(ranklist, gtItem)
        ndcg = self.getNDCG(ranklist, gtItem)
        return (hr, ndcg)

    def getHitRatio(self, ranklist, gtItem):
        for item in ranklist:
            if item == gtItem:
                return 1
        return 0

    def getNDCG(self, ranklist, gtItem):
        for i in range(len(ranklist)):
            item = ranklist[i]
            if item == gtItem:
                return math.log(2) / math.log(i+2)
        return 0

Archivo dataset.py

In [21]:
'''
Created on Aug 8, 2016
Processing datasets.

@author: Xiangnan He (xiangnanhe@gmail.com)

Modified  on Nov 10, 2017, by Lianhai Miao
'''

import scipy.sparse as sp
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader

class GDataset(object):

    def __init__(self, user_path, group_path, num_negatives):
        '''
        Constructor
        '''
        self.num_negatives = num_negatives
        # user data
        self.user_trainMatrix = self.load_rating_file_as_matrix(user_path + "Train.txt")
        print("user train")
        self.user_testRatings = self.load_rating_file_as_list(user_path + "Test.txt")
        print("user test")
        self.user_testNegatives = self.load_negative_file(user_path + "Negative.txt")
        print("user negative")
        self.num_users, self.num_items = self.user_trainMatrix.shape
        # group data
        self.group_trainMatrix = self.load_rating_file_as_matrix(group_path + "Train.txt")
        print("gr train")
        self.group_testRatings = self.load_rating_file_as_list(group_path + "Test.txt")
        print("gr test")
        self.group_testNegatives = self.load_negative_file(group_path + "Negative.txt")
        print("gr negative")

    def load_rating_file_as_list(self, filename):
        ratingList = []
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split(" ")
                user, item = int(arr[0]), int(arr[1])
                ratingList.append([user, item])
                line = f.readline()
        return ratingList

    def load_negative_file(self, filename):
        negativeList = []
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split(" ")
                negatives = []
                for x in arr[1:]:
                    negatives.append(int(x))
                negativeList.append(negatives)
                line = f.readline()
        return negativeList

    def load_rating_file_as_matrix(self, filename):
        # Get number of users and items
        num_users, num_items = 0, 0
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split(" ")
                u, i = int(arr[0]), int(arr[1])
                num_users = max(num_users, u)
                num_items = max(num_items, i)
                line = f.readline()
        # Construct matrix
        mat = sp.dok_matrix((num_users + 1, num_items + 1), dtype=np.float32)
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split(" ")
                if len(arr) > 2:
                    user, item, rating = int(arr[0]), int(arr[1]), int(arr[2])
                    if (rating > 0):
                        mat[user, item] = 1.0
                else:
                    user, item = int(arr[0]), int(arr[1])
                    mat[user, item] = 1.0
                line = f.readline()
        return mat

    def get_train_instances(self, train):
        user_input, pos_item_input, neg_item_input = [], [], []
        num_users = train.shape[0]
        num_items = train.shape[1]
        for (u, i) in train.keys():
            # positive instance
            for _ in range(self.num_negatives):
                pos_item_input.append(i)
            # negative instances
            for _ in range(self.num_negatives):
                j = np.random.randint(num_items)
                while (u, j) in train:
                    j = np.random.randint(num_items)
                user_input.append(u)
                neg_item_input.append(j)
        pi_ni = [[pi, ni] for pi, ni in zip(pos_item_input, neg_item_input)]
        return user_input, pi_ni

    def get_user_dataloader(self, batch_size):
        user, positem_negitem_at_u = self.get_train_instances(self.user_trainMatrix)
        train_data = TensorDataset(torch.LongTensor(user), torch.LongTensor(positem_negitem_at_u))
        user_train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
        return user_train_loader

    def get_group_dataloader(self, batch_size):
        group, positem_negitem_at_g = self.get_train_instances(self.group_trainMatrix)
        train_data = TensorDataset(torch.LongTensor(group), torch.LongTensor(positem_negitem_at_g))
        group_train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
        return group_train_loader

Archivo agree.py

In [19]:
'''
Created on Nov 10, 2017
Create Model

@author: Lianhai Miao
'''

import torch
import torch.nn as nn

class AGREE(nn.Module):
    def __init__(self, num_users, num_items, num_groups, embedding_dim, group_member_dict, drop_ratio):
        super(AGREE, self).__init__()
        self.userembeds = UserEmbeddingLayer(num_users, embedding_dim)
        self.itemembeds = ItemEmbeddingLayer(num_items, embedding_dim)
        self.groupembeds = GroupEmbeddingLayer(num_groups, embedding_dim)
        self.attention = AttentionLayer(2 * embedding_dim, drop_ratio)
        self.predictlayer = PredictLayer(3 * embedding_dim, drop_ratio)
        self.group_member_dict = group_member_dict
        self.num_users = num_users
        self.num_groups = len(self.group_member_dict)
        # initial model
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight)
            if isinstance(m, nn.Embedding):
                nn.init.xavier_normal_(m.weight)

    def forward(self, group_inputs, user_inputs, item_inputs):
        # train group
        if (group_inputs is not None) and (user_inputs is None):
            out = self.grp_forward(group_inputs, item_inputs)
        # train user
        else:
            out = self.usr_forward(user_inputs, item_inputs)
        return out

    # group forward
    def grp_forward(self, group_inputs, item_inputs):
        group_embeds = torch.Tensor()
        item_embeds_full = self.itemembeds(torch.LongTensor(item_inputs))
        for i, j in zip(group_inputs, item_inputs):
            members = self.group_member_dict[i.item()]
            members_embeds = self.userembeds(torch.LongTensor(members))
            items_numb = []
            for _ in members:
                items_numb.append(j)
            item_embeds = self.itemembeds(torch.LongTensor(items_numb))
            group_item_embeds = torch.cat((members_embeds, item_embeds), dim=1)
            at_wt = self.attention(group_item_embeds)
            g_embeds_with_attention = torch.matmul(at_wt, members_embeds)
            group_embeds_pure = self.groupembeds(torch.LongTensor([i]))
            g_embeds = g_embeds_with_attention + group_embeds_pure
            group_embeds = torch.cat((group_embeds, g_embeds))

        element_embeds = torch.mul(group_embeds, item_embeds_full)  # Element-wise product
        new_embeds = torch.cat((element_embeds, group_embeds, item_embeds_full), dim=1)
        y = torch.sigmoid(self.predictlayer(new_embeds))
        return y

    # user forward
    def usr_forward(self, user_inputs, item_inputs):
        user_embeds = self.userembeds(user_inputs)
        item_embeds = self.itemembeds(item_inputs)
        element_embeds = torch.mul(user_embeds, item_embeds)  # Element-wise product
        new_embeds = torch.cat((element_embeds, user_embeds, item_embeds), dim=1)
        y = torch.sigmoid(self.predictlayer(new_embeds))
        return y

class UserEmbeddingLayer(nn.Module):
    def __init__(self, num_users, embedding_dim):
        super(UserEmbeddingLayer, self).__init__()
        self.userEmbedding = nn.Embedding(num_users, embedding_dim)

    def forward(self, user_inputs):
        user_embeds = self.userEmbedding(user_inputs)
        return user_embeds


class ItemEmbeddingLayer(nn.Module):
    def __init__(self, num_items, embedding_dim):
        super(ItemEmbeddingLayer, self).__init__()
        self.itemEmbedding = nn.Embedding(num_items, embedding_dim)

    def forward(self, item_inputs):
        item_embeds = self.itemEmbedding(item_inputs)
        return item_embeds


class GroupEmbeddingLayer(nn.Module):
    def __init__(self, number_group, embedding_dim):
        super(GroupEmbeddingLayer, self).__init__()
        self.groupEmbedding = nn.Embedding(number_group, embedding_dim)

    def forward(self, num_group):
        group_embeds = self.groupEmbedding(num_group)
        return group_embeds


class AttentionLayer(nn.Module):
    def __init__(self, embedding_dim, drop_ratio=0):
        super(AttentionLayer, self).__init__()
        self.linear = nn.Sequential(
            nn.Linear(embedding_dim, 16),
            nn.ReLU(),
            nn.Dropout(drop_ratio),
            nn.Linear(16, 1),
        )

    def forward(self, x):
        out = self.linear(x)
        weight = torch.softmax(out.view(1, -1), dim=1)
        return weight


class PredictLayer(nn.Module):
    def __init__(self, embedding_dim, drop_ratio=0):
        super(PredictLayer, self).__init__()
        self.linear = nn.Sequential(
            nn.Linear(embedding_dim, 8),
            nn.ReLU(),
            nn.Dropout(drop_ratio),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        out = self.linear(x)
        return out

Ejecutamos main

In [24]:
'''
Created on Nov 10, 2017
Main function

@author: Lianhai Miao
'''

import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
from time import time

from tqdm import tqdm

config = Config()

# initial helper
helper = Helper()

# get the dict of users in group
g_m_d = helper.gen_group_member_dict(config.user_in_group_path)

# initial dataSet class
dataset = GDataset(config.user_dataset, config.group_dataset, config.num_negatives)

# get group number
num_group = len(g_m_d)
num_users, num_items = dataset.num_users, dataset.num_items

# build AGREE model
agree = AGREE(num_users, num_items, num_group, config.embedding_size, g_m_d, config.drop_ratio)

# train the model
def training(model, train_loader, epoch_id, config, type_m):
    # user trainning
    learning_rates = config.lr
    # learning rate decay
    lr = learning_rates[0]
    if epoch_id >= 15 and epoch_id < 25:
        lr = learning_rates[1]
    elif epoch_id >=20:
        lr = learning_rates[2]
    # lr decay
    if epoch_id % 5 == 0:
        lr /= 2

    # optimizer
    optimizer = optim.RMSprop(model.parameters(), lr)

    losses = []
    print('%s train_loader length: %d' % (type_m, len(train_loader)))
    for batch_id, (u, pi_ni) in tqdm(enumerate(train_loader)):
        # Data Load
        user_input = u
        pos_item_input = pi_ni[:, 0]
        neg_item_input = pi_ni[:, 1]
        # Forward
        if type_m == 'user':
            pos_prediction = model(None, user_input, pos_item_input)
            neg_prediction = model(None, user_input, neg_item_input)
        elif type_m == 'group':
            pos_prediction = model(user_input, None, pos_item_input)
            neg_prediction = model(user_input, None, neg_item_input)
        # Zero_grad
        model.zero_grad()
        # Loss
        loss = torch.mean((pos_prediction - neg_prediction -1) **2)
        # record loss history
        losses.append(loss)
        # Backward
        loss.backward()
        optimizer.step()

    print('Iteration %d, loss is [%.4f ]' % (epoch_id, torch.mean(torch.stack(losses))))


def evaluation(model, helper, testRatings, testNegatives, K, type_m):
    model.eval()
    (hits, ndcgs) = helper.evaluate_model(model, testRatings, testNegatives, K, type_m)
    hr, ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
    return hr, ndcg

# config information
print("AGREE at embedding size %d, run Iteration:%d, NDCG and HR at %d" %(config.embedding_size, config.epoch, config.topK))
# train the model
for epoch in range(config.epoch):
    agree.train()
    # 开始训练时间
    t1 = time()
    training(agree, dataset.get_user_dataloader(config.batch_size), epoch, config, 'user')

    training(agree, dataset.get_group_dataloader(config.batch_size), epoch, config, 'group')
    print("user and group training time is: [%.1f s]" % (time()-t1))
    # evaluation
    t2 = time()
    u_hr, u_ndcg = evaluation(agree, helper, dataset.user_testRatings, dataset.user_testNegatives, config.topK, 'user')
    print('User Iteration %d [%.1f s]: HR = %.4f, NDCG = %.4f, [%.1f s]' % (
        epoch, time() - t1, u_hr, u_ndcg, time() - t2))

    hr, ndcg = evaluation(agree, helper, dataset.group_testRatings, dataset.group_testNegatives, config.topK, 'group')
    print(
        'Group Iteration %d [%.1f s]: HR = %.4f, '
        'NDCG = %.4f, [%.1f s]' % (epoch, time() - t1, hr, ndcg, time() - t2))


print("Done!")

AGREE at embedding size 32, run Iteration:30, NDCG and HR at 5
user train_loader length: 13997


1661it [07:52,  3.52it/s]


KeyboardInterrupt: 